In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from os.path import exists
sns.set_style("white")
%matplotlib inline

---

# <center>FHHPS - Data Cleaning</center>

This is the annotated code used for cleaning our data.

Information about the raw data can be found <a href="http://catalog.ihsn.org/index.php/catalog/4522/related_materials">here</a>. 

Prior to running this code, used the read-in programs for ASI data created by Olivier Duprez and available at the International Household Survey Network (<a href="http://catalog.ihsn.org/index.php/catalog/central">link</a>). 

### Checking if all files are in place

Files should be in a certain structure for this code to work. The next two cells make sure that's the case.

In [4]:
PATH_TO_ASI_FILES = "ASI_FILES/" # BLx200y.dta files should be in this directory
PATH_TO_ISIC = "ISIC/"      
PATH_TO_DEFLATORS = "Deflators/"

If any of the cells below raise an error, files are *not* in the correct structure. 

In [5]:
ASI_FILES = ['BLA2008.dta', 'BLA2009.dta', 'BLB2008.dta', 'BLB2009.dta', 'BLC2008.dta',
 'BLC2009.dta', 'BLD2008.dta', 'BLD2009.dta', 'BLE2008.dta', 'BLE2009.dta', 'BLF2008.dta',
 'BLF2009.dta', 'BLG2008.dta', 'BLG2009.dta', 'BLH2008.dta', 'BLH2009.dta',
 'BLI2008.dta', 'BLI2009.dta', 'BLJ2008.dta', 'BLJ2009.dta']

ISIC_FILES = ['ISIC3-ISIC2.txt', 'ISIC3-ISIC31.txt', 'ISIC4-ISIC31.txt']

deflator_files = ['final input deflator.dta',
 'final output deflator.dta',
 'gdp deflator national.csv',
 'RBI Capital Formation Accounts.csv']

assert exists(PATH_TO_ASI_FILES)
for f in ASI_FILES:
    assert exists(PATH_TO_ASI_FILES + f)

    
assert exists(PATH_TO_ISIC)
for f in ISIC_FILES:
    assert exists(PATH_TO_ISIC + f)
    
for f in deflator_files:
    assert exists(PATH_TO_DEFLATORS + f)
    
print("All files in place.")

All files in place.


---

## Overview

Broadly, we follow these steps:
    
1) <b>Read data from ASI files.</b> 

There are 10 files for each year, named `BLA2008, BLA2009, BLB2008, BLB2009`, and so on. (`BL` stands for `BLOCK`).

The `BLA` files have firms' identification details such as factory ID, state, sampling scheme, etc.

The `BLB` to `BLJ` files have information on firms' fixed assets, inputs, expenses, output, etc.

2) <b>Merge in auxiliary information: ISIC and deflators</b> 

We must deflate monetary values to 2005 values. 

For that, we would like to use Allcott et al's deflators. However, while Allcott et al provide industry-specific deflators, they are associated with 1987's industry codes. So we must also map current industry codes, named NIC2008 and NIC2004, back to NIC1987. In order to do that, we merge in additional information on the International Standard Industry Classification schemes, which are equivalent to India's National Industry Classification.

3) <b>Produce an intermediate sample by dropping unreasonable or missing observations</b>

Here we follow Allcott's data cleaning methodology. See below for details.

We also drop all factories that appear only in one of the two years in our sample.

----

# Step 1: Extract ASI data

## ASI BLOCKS B-J

Auxiliary Functions

In [6]:
def extract(letter, condition_column, condition_value, columns_to_keep, new_column_names):
    
    """
    Extracts the data from files BLx200y, where x is a letter and y 8 and 9
    """
    
    output = []
    
    columns_to_keep = [columns_to_keep] if isinstance(columns_to_keep, str) else columns_to_keep
        
    # Always keep FACT_ID and YEAR
    columns_to_keep = ["FACT_ID", "YEAR"] + columns_to_keep    
    
    for year in ["2008", "2009"]:
        
        # Read in data
        filename = PATH_TO_ASI_FILES + "/BL%s%s.dta" % (letter, year)
        df = pd.read_stata(filename)
        
        # Set condition
        if isinstance(condition_value, str):
            condition = df[condition_column].str.contains(condition_value)
        else:
            condition = df[condition_column] == condition_value 
        condition = condition.fillna(False)

        # Keep only relevant rows and columns
        df = df.loc[condition, columns_to_keep]
        
        # Append to list
        output.append(df)
    
    # Put everything together after grabbing information for each year
    output = pd.concat(output).set_index(["FACT_ID", "YEAR"]).sort_index()
    
    if isinstance(output, pd.DataFrame):
        output.columns = [new_column_names] if isinstance(new_column_names, str) else [new_column_names]
    else:
        output.columns = new_column_names
        
    return output

In [7]:
def pad_zeros(x):
    return "%04d" % int(x)

### Output

Output is defined as the gross sales value in Rupees.

<b>Details:</b>

> The <b>gross sale value</b> of the products as charged from the customers will be reported here. It includes excise duty paid or sales tax realized by the factory on behalf of the Government as also all distributive expenses incurred such as (i) discount or rebate, allowances for returnable cases or other packing and any other drawback allowed to customers, (ii) charges for carriage, outward, and (iii) commission to selling agents. 


<b>Data:</b> 

Column J_I7 (GrossValue), Rows where E_I1 (Field) == "Total"

<b>Allcott equivalent</b>

productTotalGrossValue -> grsale

In [8]:
Y = extract(letter = "J", 
            condition_column = "J_I1",
            condition_value = 12,
            columns_to_keep = "J_I7",
            new_column_names = "Y")

### Capital


Capital is defined as the net value of fixed assets (defined below) in Rupees. There are two such variables.

+ `Opening On - Net Value` refers to Net Value on 01-04-2008 (2007); while 

+ `Closing On - Net Value` refers to Net Value on 31-03-2009 (2008).

<b>Details:</b>

> <b>Fixed assets</b> are those, which have generally normal productive life of more than one year; it covers all type of assets, new or used or own constructed, deployed for productions, transportation, living or recreational facilities, hospitals, schools, etc. for factory personnel; it would include land, building, plant and machinery, transport equipment, etc.; it includes the fixed assets of the head office allocable to the factory and also the full value of assets taken on hire-purchase basis (whether fully paid or not) excluding interest element; it excludes intangible assets and assets solely used for post-manufacturing activities such as, sale, storage, distribution, etc.


<b>Data:</b> 

Columns C_I12 (Opening on - Net Value) and C_I13 (Closing on - Net Value), where C_I1 (Field) == "Total"

<b>Allcott equivalent</b>

+ `fixedTotalNetOpen` -> `fcapopen`

+ `fixedTotalNetClose` -> `fcapclose`

<b>Notes</b>

Allcott uses `fcapclose` in his regressions. 

In [9]:
K = extract(letter = "C", 
            condition_column = "C_I1",
            condition_value = "Total",
            columns_to_keep = ["C_I12", "C_I13"],
            new_column_names = ["K_open", "K_close"])

In [10]:
K.head()

K_open    K_close
FACT_ID  YEAR                      
0000133F 2009   384241.0   644280.0
0000206F 2008  8023505.0  8232079.0
         2009  8232079.0  8881984.0
0000208B 2008    62338.0    54401.0
         2009    54401.0    49904.0

### Labor  and Wages

Labor is defined as the average number of persons worked.

Wages are total emoluments (see below) paid to employees throughout the year, in Rupees.

<b>Data:</b> 

+ `Labor`: Columns E_I6 (Average Number of persons worked), where E_I1 (Field) includes "Total employees"

+ `Wages`: Columns E_I8 (Wages/salaries ), where E_I1 (Field) includes "Total employees"


<b>Details:</b>

*Emoluments* are defined as wages paid to all employees plus imputed value of benefits in kind, i.e., the net cost to the employers on those goods and services provided to employees free of charge or at markedly reduced cost which are clearly and primarily of benefit to the employees as consumers. It includes profit sharing, festival and other bonuses and ex-gratia payments paid at less frequent intervals (i.e. other than bonus paid more or less regularly for each period). Benefits in kind include supplies or services rendered such as housing, medical, education and recreation facilities. Personal insurance, income tax, house rent allowance, conveyance, etc. for payment by the factory also is included in the emoluments. 

<b>Allcott equivalent</b>

+ Labor: empTotalAverage -> totpersons

In [11]:
LW = extract(letter = "E", 
            condition_column = "E_I1",
            condition_value = "Total employees",
            columns_to_keep = ["E_I6", "E_I8"],
            new_column_names = ["L", "W"])

In [12]:
LW.head()

L          W
FACT_ID  YEAR                
0000133F 2009   10   887775.0
0000206F 2008  167  7441445.0
         2009  128  8020466.0
0000208B 2008  244  4162863.0
         2009  253  4250227.0

### Materials

Materials are the sum of the value of imported and indigenous goods (raw materials, components, chemicals, packing material, etc.) which entered into the production process of the factory during the accounting year, in Rupees.

<b>Data:</b> 

+ Imported: Columns I_I6 (purchase value of imported inputs) where I_I1 (Field) == 7 (representing "total")

+ Indigenous: Columns H_I6 (purchase value of indigenous inputs) where H_I1 (Field) == "Total"

<b>Allcott equivalent</b>

None

<b>Notes</b>

Allcott has a variable called `matls`, but his definition of materials excludes electricity and fuel. In our analysis both electricity and fuel and lumped with other inputs.

In [13]:
# Imported materials
M1 = extract(letter = "I", 
             condition_column = "I_I1",
             condition_value = 7,
             columns_to_keep = ["I_I6"],
             new_column_names = ["M1"])

# Indigenous_materials
M2 = extract(letter = "H", 
             condition_column = "H_I1",
             condition_value = "\\bTotal\\b", # Make sure not to match TotalBasic, etc
             columns_to_keep = ["H_I6"],
             new_column_names = ["M2"])

M = pd.DataFrame(pd.concat([M1, M2], axis = 1).fillna(0).sum(1))
M.columns = ["M"]

In [14]:
M.head()

M
FACT_ID  YEAR            
0000133F 2009    720617.0
0000206F 2008  19017192.0
         2009  19527426.0
0000208B 2008   3312668.0
         2009   5102722.0

Join every variable by "FACT_ID" and "YEAR".

In [12]:
df = pd.concat([Y, K, LW, M], axis = 1, join = "outer")

In [13]:
df.to_csv("blocks_B-J.csv")

----

## Factory information (ASI BLOCK A)

Here we extract data from ASI Block A (Identification particulars).

+ `FACT_ID` and `YEAR` follow the same conventions as above.

+ `state` is encoded in digits 5-6 in `FACT_ID`

+ `scheme` refers to *Census* or *Sample* scheme

+ `urban` is recoded as dummy

In [14]:
output = []
for year in ["2008", "2009"]:
    df = pd.read_stata(PATH_TO_ASI_FILES + "BLA%s.dta" % year)
    df["state"] = df["FACT_ID"].apply(lambda x: x[5:7])
    df = df.drop_duplicates(subset = "FACT_ID").set_index(["FACT_ID", "YEAR"])
    open_factory = df["A12"] == "Open"
    df = df.loc[open_factory, ["state", "A3", "A5", "A9"]]
    df["A9"] = df["A9"].replace("Rural", 0).replace("Urban", 1)
    output.append(df)

In [15]:
info = pd.concat(output).sort_index()
info.columns = ["state", "scheme", "nic5", "urban"]

In [16]:
info.to_csv("info.csv")

---

# Step 2: Merge in ISIC and Deflators

## ISIC

In order to revert NIC2004 and NIC2008 codes back to 1987, we use correspondence tables provided in the <a href="http://unstats.un.org/unsd/cr/registry/regdnld.asp?Lg=1">UN Statistics Division</a> website

+ ISIC3.1-ISIC4.zip <a href="http://unstats.un.org/unsd/cr/registry/regdntransfer.asp?f=121">link</a> (Equivalent to NIC2004 - NIC2008)
+ ISIC3-ISIC3.1.zip <a href="http://unstats.un.org/unsd/cr/registry/regdntransfer.asp?f=39">link</a> (Equivalent to NIC 1998 - NIC2004)
+ ISIC2-ISIC3.zip <a href="http://unstats.un.org/unsd/cr/registry/regdntransfer.asp?f=68">link</a> (Equivalent to NIC1987 - NIC1998)

In [17]:
isic2_3 = pd.read_table(PATH_TO_ISIC + "ISIC3-ISIC2.txt", sep = ",")[["ISIC3", "ISIC2"]].applymap(pad_zeros)
isic3_31 = pd.read_table(PATH_TO_ISIC + "ISIC3-ISIC31.txt", sep = ",", na_values = "n/a")[["Rev3", "Rev31"]].dropna().applymap(pad_zeros)
isic3_31.columns = ["ISIC3", "ISIC31"]
isic31_4 = pd.read_table(PATH_TO_ISIC + "ISIC4-ISIC31.txt", sep = ",", na_values = "n/a")[["ISIC31code","ISIC4code"]].applymap(pad_zeros)
isic31_4.columns = ["ISIC31", "ISIC4"]
isic = pd.merge(isic2_3, isic3_31, on = "ISIC3")
isic = pd.merge(isic, isic31_4, on = "ISIC31")
isic.to_csv("isic_correspondences.csv", index = False)

---

## <b>Deflators</b>

We deflate output, material inputs and capital using Allcott's deflators.

We also deflate wages using a national deflator, also from Allcott's files.

From Allcott's appendix

> All financial amounts are deflated to constant 2004-05 Rupees. <b>Revenue</b> (gross sales) is deflated by a three-digit commodity price deflators as available in the commodity-based table “Index Numbers Of Wholesale Prices In India – By Groups And Sub-Groups (Yearly Averages)” produced by the Office of the Economic Adviser-Ministry of Commerce & Industry.2 Each three-digit NIC-1987 code is assigned to a commodity listed in this table. The corresponding commodity de- flator is used to deflate revenues. To deflate <b>material inputs</b>, we construct the average output deflator of a given industry’s supplier industries based on India’s 1993-94 input-output table, available from the Central Statistical Organization. (...) <b>Capital</b> is deflated by an implied national deflator calculated from “Table 13: Sector-wise Gross Capital Formation” from the Reserve Bank of India’s Handbook of Statistics on the Indian Economy.3 Electricity costs are deflated using a national GDP deflator.


#### Comments

1) For future reference, the relevant input-output table for the years 2007-2008 can be found <a href="http://mospi.nic.in/sites/default/files/reports_and_publication/cso_national_accounts/input_output_transactions_table/2007_08/Binder1%201.pdf">here</a>.

2) Allcott has a "state gdp deflator" in his code. It's not clear how it's used, and it's not among the available files in this website.

### &#9760; IMPORTANT: No NIC-specific deflators for non-manufacturing firms

Allcott only has input and output deflators for manufacturing firms. For other kinds of firms, we impute the overall average for that year.


In [18]:
deflator_years = [2007, 2008]

The next columns read Allcott deflator files and grab the information pertinent to years 2007 and 2008.

In [19]:
input_deflator = pd.read_stata(PATH_TO_DEFLATORS + "final input deflator.dta")
input_deflator = input_deflator[input_deflator["year"].isin(deflator_years)]
input_deflator["input_deflator"] /= 100 

In [20]:
output_deflator = pd.read_stata(Deflators + "final output deflator.dta")
output_deflator = output_deflator[output_deflator["year"].isin(deflator_years)]
output_deflator.rename(columns = {"deflator": "output_deflator"}, inplace = True)
output_deflator["output_deflator"] /= 100 

In [21]:
national_deflator = pd.read_csv(PATH_TO_DEFLATORS + "gdp deflator national.csv")
national_deflator = national_deflator[national_deflator["year"].isin(deflator_years)]
national_deflator.rename(columns= {"gdp_defl_natl":"national_deflator"}, inplace = True)
national_deflator["national_deflator"] /= 100

In [22]:
capital_deflator = pd.read_csv(PATH_TO_DEFLATORS + "RBI Capital Formation Accounts.csv")
capital_deflator["Year"] = capital_deflator["Year"].apply(lambda x: int(str(x)[:4]))
capital_deflator = capital_deflator[capital_deflator["Year"].isin(deflator_years)][["Year","Capital_Deflator"]]
capital_deflator.columns = ["year", "capital_deflator"]

In [23]:
deflator = pd.merge(input_deflator, national_deflator)
deflator = pd.merge(deflator, output_deflator)
deflator = pd.merge(deflator, capital_deflator)

In [24]:
deflator.rename(columns = {"nic3digit": "NIC1987", "year":"YEAR"}, inplace = True)

In [25]:
deflator["YEAR"] += 1 # To agree with main data set 

In [26]:
deflator.to_csv("deflators.csv")

In [27]:
deflator.head(3)

,NIC1987,YEAR,input_deflator,national_deflator,output_deflator,capital_deflator
0,200,2008,1.108265,1.173171,1.10197,1.147185
1,201,2008,1.108265,1.173171,1.17331,1.147185
2,202,2008,1.108265,1.173171,1.10197,1.147185


----

## Merging

In [28]:
info = pd.read_csv("info.csv")                # Extracted from ASI dataset block A
blocks_bj = pd.read_csv("blocks_B-J.csv")     # Extracted from ASI dataset blocks B-J
deflator = pd.read_csv("deflators.csv")       # Allcott's deflator files
isic = pd.read_csv("isic_correspondences.csv").applymap(str)

Merge ISIC to Block-A information

In [29]:
info["nic4"] = info["nic5"].apply(lambda x: str(x)[:4])

In [30]:
info8 = pd.merge(info[info["YEAR"] == 2008], 
                 isic, 
                 left_on = "nic4", 
                 right_on = "ISIC31",
                 how = "left")

In [31]:
info9 = pd.merge(info[info["YEAR"] == 2009], 
                 isic, 
                 left_on = "nic4", 
                 right_on = "ISIC4",
                 how = "left")

### &#9760; IMPORTANT  NIC concordance information is insufficient
 	

We are trying to replicate this from Allcott's appendix:

> [NIC] classifications were revised in 1987, 1988, 2004 and 2008. We convert all industry classifications to the NIC-1987 scheme using concordances provided by MOSPI [India Ministry of Statistics and Programme Implementation]

(Note: we would not need to bother with this, if it weren't for the fact that Allcott's input and output deflators use NIC1987 information.)

The concordance table mentioned by Allcott is probably <a href="http://udyogaadhaar.gov.in/ua/Document/nic_2008_17apr09.pdf">this</a> one (see page 121).


One issue we face here is that the table contains only partial information there about how to merge NIC2004 and NIC2008. For example, using the table we know the following mapping:

| NIC2004 |  NIC2008 |
|:-:|:-:|
| 3699  | 1512  |
| 1912  | 1512  |
| 3699  | 1629  |
| 3699  | 1709  |
| 3699  | 3212  |
| 3699  | 3290  |


That is, part of what was 3699 in NIC2004 now corresponds to 1512 in NIC2008, and the rest comes from 1921. Moreover, some of 3699 ended up in 1629, 1709, etc. 

However, we do not know which parts of 3699 went to which code, and we don't know which parts of 1512 came from 3699 and which came from 1912.

<b>At the moment, we are artificially creating a one-to-one mapping by choosing one of the NICs at random.</b>

Allcott does pretty much the same: there are files named (e.g.) `NIC0408Crosswalk_4digit.dta` that seem to provide a one-to-many mapping from NIC2004 to NIC2008. However, they are also imperfect mappings. For example, they will map NIC2004 3699 into NIC2008 3290 and 3212 categories, but it will ignore the possibility that 3699 might go into 1512, 1629 and 3290. 

So we could instead use Allcott's mappings, but they are not perfect either.

In [32]:
info8 = info8.drop_duplicates("FACT_ID") 
info9 = info9.drop_duplicates("FACT_ID")

In [33]:
info = pd.concat([info8, info9])
info["NIC1987"] = info["ISIC2"].apply(lambda x: str(x)[:3])

In [34]:
nans = info["ISIC4"].isnull().sum()

In [35]:
info[["ISIC3","ISIC2","ISIC31","ISIC4"]] = info[["ISIC3","ISIC2","ISIC31","ISIC4"]].fillna("9999")
info["NIC1987"] = info["NIC1987"].fillna("9999")

In [36]:
print("Filled", nans, "missing industry codes with 9999.")

Filled 2705 missing industry codes with 9999.


Merge in variables extracted from blocks B-J.

In [37]:
df = pd.merge(blocks_bj, info, on= ["FACT_ID", "YEAR"])

#### Merge deflators

Recall that deflators only exist for manufacturing NICs.

In [38]:
deflator["NIC1987"] = deflator["NIC1987"].astype(str)
df = pd.merge(df, deflator, on= ["YEAR", "NIC1987"], how = "left")
df.shape

(76594, 23)

For non-manufacturing NICs, impute average deflator for that year.

In [39]:
for defl in ["input_deflator", "national_deflator", "output_deflator", "capital_deflator"]:
    for year in [2008, 2009]:
        avg_defl = df.loc[df["YEAR"] == year, defl].mean()
        df[defl] = df[defl].fillna(avg_defl)

In [40]:
df.to_csv("merged.csv") # Just to checkpoint
print(df.shape)

(76594, 23)


In [41]:
df.head(3)

,FACT_ID,YEAR,Y,K_open,K_close,L,W,M,state,scheme,...,ISIC3,ISIC2,ISIC31,ISIC4,NIC1987,Unnamed: 0,input_deflator,national_deflator,output_deflator,capital_deflator
0,0000133F,2009,2175183.0,384241.0,644280.0,10.0,887775.0,720617.0,33,Sample,...,2695,3699,2695,2395,369,343.0,1.231560,1.274822,1.013428,1.229909
1,0000206F,2008,33497412.0,8023505.0,8232079.0,167.0,7441445.0,19017192.0,6,Census,...,2423,3522,2423,2100,352,144.0,1.322133,1.173171,1.172029,1.147185
2,0000206F,2009,32533041.0,8232079.0,8881984.0,128.0,8020466.0,19527426.0,6,Census,...,1721,3212,1721,3250,321,298.0,1.389533,1.274822,1.090890,1.229909


---

# Step 3) Clean and produce intermediate sample

In [42]:
df = pd.read_csv("merged.csv")

In [43]:
n = df.shape[0]
print("Initial number of observations: ", n)

Initial number of observations:  76594


<font color="red">Drop</font> if no information on output

In [44]:
df = df.dropna(subset = ["Y"])
print("Dropped observations:", n - df.shape[0])
n = df.shape[0]

Dropped observations: 12127


<font color="red">Drop</font> if unrealistic count of workers

In [45]:
df = df[df["L"] < 100000]

In [46]:
print("Dropped observations:", n - df.shape[0])
n = df.shape[0]

Dropped observations: 43


<font color="red">Drop</font> materials, capital or wages are more than twice the output

In [47]:
df = df[df["M"]/df["Y"] < 2]
df = df[df["K_close"]/df["Y"] < 2]
df = df[df["K_open"]/df["Y"] < 2]
df = df[df["W"]/df["Y"] < 2]

In [48]:
print("Dropped observations:", n - df.shape[0])
n = df.shape[0]

Dropped observations: 4442


<font color="red">Drop</font> if not there for both years.

In [49]:
both_years = df.groupby("FACT_ID").size() 
both_years = both_years[both_years == 2].index
df = df.reset_index()
df = df.loc[df["FACT_ID"].isin(both_years)].set_index(["FACT_ID", "YEAR"])

In [50]:
print("Dropped observations:", n - df.shape[0])
n = df.shape[0]

Dropped observations: 32282


<font color="blue">Trim</font> regressors at zero

In [51]:
regressors = ["M","K_close","K_open","Y","W","L"]
n_negs = (df[regressors] < 0).sum()

In [52]:
for r in regressors:
    df.loc[df[r] < 0, r] = 0

In [53]:
print("Changed values")
n_negs

Changed values


M          0
K_close    0
K_open     2
Y          0
W          0
L          0
dtype: int64

<font color="blue">Fill missing values</font> for regressors with zero

In [54]:
print("Changed values")
df[regressors].isnull().sum()

Changed values


M          0
K_close    0
K_open     0
Y          0
W          0
L          0
dtype: int64

In [55]:
df[regressors] = df[regressors].fillna(0)

<font color="blue">Fill missing values</font> for NIC1987 with 999

In [56]:
print("Changing", df["NIC1987"].isnull().sum(), "values")
df["NIC1987"].fillna("999", inplace = True)

Changing 862 values


In [57]:
df.to_csv("clean.csv") # Another checkpoint

---

### Deflating

In [58]:
df = pd.read_csv("clean.csv")

Deflate to 2004-05 prices

In [59]:
df["Y"] /= df["output_deflator"]
df["M"] /= df["input_deflator"]
df["K_open"] /= df["capital_deflator"]
df["K_close"] /= df["capital_deflator"]
df["W"] /= df["national_deflator"]

---

### Construct variables

In [60]:
df["K_W"] = df["K_open"] / df["W"]
df["YM_W"] = (df["Y"] - df["M"]) / df["W"]

In [61]:
keep = ["NIC1987", "state", "scheme","urban","Y", "K_close", "L", "M", "W", "K_W", "YM_W"]
index = ["FACT_ID", "YEAR"]
df = df.set_index(index)[keep] 

<font color="Darkgreen">Create</font> <i>panelgroup</i> variable that merges state and the first two digits of NIC1987.

This is to mimic Allcott's panelgroup variable as best as we can. Allcott actually used "superNICs". 

In [62]:
df["panelgroup"] = df["NIC1987"].astype(int).astype(str).apply(lambda x: x[:2]) + df["state"].astype(str)

In [63]:
df.to_csv("intermediate.csv")

----

##  &#9760; We are NOT dropping obs with extreme TFP

Following Allcott's appendix:

> With this intermediate sample, we use median regression to estimate revenue productivity (TFPR) under a full Cobb-Douglas model in capital, labor, materi- als, and energy and assuming constant returns to scale. This full Cobb-Douglas revenue productivity term is used only for the final sample restriction, which is to drop 4,521 plant-years which have log-TFPR greater than 3.5 in absolute value from the sample median. Such outlying TFPR values strongly suggest misreported inputs or revenues.

This is done by  the Allcott's "subroutine" `Estimate median alphas_qreg_supernic_Nov2014.do`. However, the code is a little convoluted, and they also use information about supernics to run their regressions, so we could not follow their code even if we wanted to.

Instead, I wrote an R script named `qregs.R` that uses the R library <a href="http://rqpd.r-forge.r-project.org/">rqpd</a>, which implements <a href="http://www.sciencedirect.com/science/article/pii/S0047259X04001113">Koenker's 2004</a> *quantile regression for longitudinal data* with and a dummy for the year, and fixed effects for the variable `panelgroup`, defined above. It loads the dataset `intermediate.csv`  runs the regression explained in the previous paragraph, trims the data set at the lower (1%) and upper (3%), and finally saves a new file named `trimmed_intermediate.csv`. 

In [64]:
#!R < qregs.R --no-save  # To run this R code, uncomment me (but don't erase the initial exclamation mark)

---

## Final computations

Almost there. We will compute the following two new variables.

<center>

`YM_W` = $\log(\frac{Y_{i} - M_{i}}{W_{i}})$
<br><br>

`K_W` = $\log(\frac{K_i}{W_{i}})$

</center>

In [65]:
df = pd.read_csv("intermediate.csv")
#df = pd.read_csv("trimmed_intermediate.csv") # Uncomment this to use the trimmed data

<font color="red">Drop</font> if Y <= M (Cannot take logs!)

In [66]:
df = df[df["YM_W"] > 0]

Take logs

In [67]:
with np.errstate(divide="ignore", invalid="ignore"): # Suppress errors here
    df[["YM_W", "K_W"]] = df[["YM_W", "K_W"]].applymap(lambda x: np.log(x))

In [68]:
df = df.reset_index()

Now pivot this table to make different years side-by-side. When dealing with the "scheme" column, if the firm has "Census" in one year but "Sample" in the other, we keep it as "Sample". Firms that belong only to the Census scheme may not be representative of the industry population. 

More information about different schemes <a href="http://mail.mospi.gov.in/index.php/catalog/164/sampling">here</a>.

In [69]:
def has_sample(sch):
    if np.any(sch == "Sample"):
        return "Sample"
    else:
        return "Census"

In [78]:
# Pivot table
variables = [df.pivot(index = "FACT_ID", 
                           columns = "YEAR", 
                           values = v) 
                          for v in ["K_W", "YM_W"]]

scheme = df.groupby("FACT_ID")["scheme"].apply(has_sample)

df_clean = pd.concat(variables + [scheme], axis = 1).dropna()

Renaming

In [79]:
df_clean.columns = ["X1", "X2", "Y1", "Y2", "scheme"]

<font color="red">Drop</font> if the procedure above led to an `inf` or a `nan`.

In [81]:
with pd.option_context('mode.use_inf_as_null', True):
    n = df_clean.shape[0]
    df_clean = df_clean.dropna() # Cleans observation whose log was inf or nan
    print("Dropped", n - df_clean.shape[0], "observations.")

Dropped 0 observations.


In [82]:
df_clean.describe()

,X1,X2,Y1,Y2
count,12753.000000,12753.000000,12753.000000,12753.000000
mean,0.996419,1.016154,1.867388,1.845008
std,1.471070,1.468719,1.055174,1.076660
min,-14.946987,-15.875411,-4.762876,-7.121545
25%,0.156805,0.198829,1.249176,1.205268
50%,1.101125,1.128113,1.850216,1.834184
75%,1.949576,1.979141,2.485937,2.476799
max,5.474800,6.740813,6.499139,9.745045


We finally stop here and save the file.

In [73]:
df_clean.to_csv("indian_data.csv")

----

### Restricted sample

Keeping only the "sample" scheme.

In [74]:
df_clean.head()

,X1,X2,Y1,Y2,scheme
FACT_ID,,,,,
0000206F,0.097710,0.061909,0.805675,0.918872,Census
0000208B,-4.178988,-4.322479,0.595036,0.109149,Census
0000706F,1.096929,1.197228,1.540723,1.271278,Sample
0000733F,-1.883351,-1.958287,1.045614,1.094915,Sample
0000806F,1.007469,1.285659,2.208538,1.981069,Census


In [75]:
valid = df_clean["scheme"] == "Sample"
df_sample = df_clean.loc[valid]
df_sample.to_csv("indian_data_sample_only.csv")

In [76]:
df_sample.describe()

,X1,X2,Y1,Y2
count,3665.000000,3665.000000,3665.000000,3665.000000
mean,0.925723,0.922483,1.873001,1.788654
std,1.469873,1.478046,1.068612,1.136582
min,-14.946987,-15.875411,-3.343289,-4.925899
25%,0.124854,0.150418,1.216394,1.142020
50%,1.012952,1.010290,1.820004,1.762743
75%,1.842254,1.842429,2.483442,2.438124
max,5.137378,5.290437,5.813139,9.745045


In [77]:
df_sample.describe()

,X1,X2,Y1,Y2
count,3665.000000,3665.000000,3665.000000,3665.000000
mean,0.925723,0.922483,1.873001,1.788654
std,1.469873,1.478046,1.068612,1.136582
min,-14.946987,-15.875411,-3.343289,-4.925899
25%,0.124854,0.150418,1.216394,1.142020
50%,1.012952,1.010290,1.820004,1.762743
75%,1.842254,1.842429,2.483442,2.438124
max,5.137378,5.290437,5.813139,9.745045


Done (🎉).